### Train/Test Split

In [ ]:
# import packages
import numpy as np
import pandas as pd
import librosa
import librosa.display as dsp 
import os 
from tqdm import tqdm
import glob
import random
import collections
import shutil

In [ ]:
os.chdir('/Users/ignat/Documents/UC Davis/Fall 2022/ECS 271/Project')

In [ ]:
# read in the list of the images
image_paths = []
path = 'spectogram_images'
for data_path in glob.glob(path + '/*/*'):
    image_paths.append(data_path)

# shuffle the images
random.shuffle(image_paths)

In [ ]:
def copy_img(img_paths, dest_folder):
  # double check the destination root folder exists
  if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

  # move each image
  for img in tqdm(img_paths):
    img_name = img.split(os.path.sep)[-1]
    img_label = img.split(os.path.sep)[-2]
    label_folder = os.path.join(dest_folder, img_label)

    # check if destination label folder exists 
    if not os.path.exists(label_folder):
      os.makedirs(label_folder)
  
    # copy over the file
    destination = os.path.join(label_folder, img_name)
    shutil.copy(img, destination)

In [ ]:
def split_train_test(img_paths, train_dest_folder, test_dest_folder, split=0.8):
  # split into training and testing
  train_img_paths, test_img_paths = img_paths[:int(split*len(img_paths))], img_paths[int(split*len(img_paths)):] 
  
  copy_img(train_img_paths, train_dest_folder)
  copy_img(test_img_paths, test_dest_folder)

In [ ]:
# split images
split_train_test(image_paths, 'data/train', 'data/test', 0.8)

In [ ]:
# check the distribution of digits in train and test
def check_dist(path):
    image_paths = []
    for data_path in glob.glob(path + '/*/*'):
        image_paths.append(data_path)
    
    counter = collections.Counter([x.split('/')[-2] for x in image_paths])
    return(counter)

### Model Training

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import os
import PIL
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [ ]:
os.chdir('/Users/ignat/Documents/UC Davis/Fall 2022/ECS 271/Project')

In [ ]:
# batch size
BATCH_SIZE = 64

In [ ]:
# the training transforms
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((227,227)),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])
# the validation transforms
valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((227,227)),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

In [ ]:
# training dataset
train_dataset = datasets.ImageFolder(
    root='data/train',
    transform=train_transform
)

# training data loaders
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=4, pin_memory=True
)

In [ ]:
# validation dataset
valid_dataset = datasets.ImageFolder(
    root='data/test',
    transform=valid_transform
)

# validation data loaders
valid_loader = DataLoader(
    valid_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=4, pin_memory=True
)

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 1024),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1024, 1024),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(1024, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
num_classes = 10
num_epochs = 90
batch_size = 64
learning_rate = 0.001

model = AlexNet(num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

# Train the model
total_step = len(train_loader)

In [ ]:
# helper functions
def save_model(epochs, model, optimizer, criterion):
    """
    Function to save the trained model to disk.
    """
    torch.save({
                'epoch': epochs,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, 'model.pth')

In [ ]:

# store loss
loss_tracker = []

# store accuracy 
test_acc_tracker = []
train_acc_tracker = []

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images
        labels = labels
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    loss_tracker.append(running_loss)

    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images
            labels = labels
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

    #print('Accuracy on validation images: {} %'.format( 100 * correct / total)) 
    test_acc_tracker.append(correct / total)

    # Testing set accuracy
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in train_loader:
            images = images
            labels = labels
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

    train_acc_tracker.append(correct / total)

    # save model
    save_model(epoch, model, optimizer, criterion)

# 

In [ ]:
# loss
plt.plot(np.arange(0,len(loss_tracker)), loss_tracker)
plt.title('Loss Over the Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:

# accuracy
plt.plot(np.arange(0,len(test_acc_tracker)), test_acc_tracker)
plt.title('Testing Accuracy Over the Epochs')
plt.xlabel('Epoch')
plt.ylabel('% Accuracy')

In [ ]:
# get all ground truths and predicted labels
with torch.no_grad():
    y_pred = []
    y_true = []
    for images, labels in tqdm(valid_loader):
        labels = labels
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.append(predicted.tolist())
        y_true.append(labels.tolist())


In [ ]:
y_true = [item for sublist in y_true for item in sublist]
y_pred = [item for sublist in y_pred for item in sublist]

In [ ]:
target_names = [str(x) for x in range(10)]
output_dict = classification_report(y_true, y_pred, output_dict = True)
df_temp = pd.DataFrame(output_dict).transpose()
df_temp